

<figure>
   <IMG SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">
</figure>
    
# NLmod examples    

This example notebook shows some examples of the `nlmod` package. This package contains functions to create modflow models anywhere in the Netherlands.

### Table of contents
1. [Create model](#create)
2. [Run model](#run)
3. [Visualise](#Visualise)
4. [Local grid refinement](#local)

In [1]:
import matplotlib.pyplot as plt
import flopy
import os
import geopandas as gpd

import nlmod

print(f'nlmod version: {nlmod.__version__}')

nlmod version: 0.0.1b


### 4. Local grid refinement<a name="local"></a>

Modflow 6 makes it possible to use locally refined grids. In NLmod you can use a shapefile and a number of levels to specify where and how much you want to use local grid refinement. Below we use a shapefile of the Planetenweg in IJmuiden and set the refinement levels at 2. This well create a grid with cells of 100x100m except at the Planetenweg where the cells will be refined to 25x25m. See also figures below.

Note: the `gen_model_unstructured` function also returns the `gridprops` dictionary. This contains data that is necesary to create some packages. It can be time consuming to generate this data.

In [2]:
# model settings unstructured
model_ws = 'model2'
model_name = 'IJm_planeten'
refine_shp_fname = os.path.join(nlmod.nlmod_datadir, 'shapes', 'planetenweg_ijmuiden')
levels=2
use_cache=False
extent=[95000., 105000., 494000., 500000.]
delr=100.
delc=100.
steady_state=False
steady_start=True
transient_timesteps=5
perlen=1.
verbose=True
start_time='2015-1-1'
gridtype = 'unstructured'
use_regis=True
regis_botm_layer=b'MSz1'
use_geotop=True
add_northsea=True
starting_head=1.0

In [3]:
# Model directories
figdir2, cachedir2, gridgen_ws = nlmod.util.get_model_dirs(model_ws, gridtype=gridtype)


# create model time dataset
model_ds2 = nlmod.mtime.get_model_ds_time(model_name, model_ws, start_time,
                                         steady_state, steady_start=steady_start,
                                         transient_timesteps=transient_timesteps,
                                         perlen=perlen)

# create modflow packages
sim, gwf2 = nlmod.mfpackages.sim_tdis_gwf_ims_from_model_ds(model_ds2,
                                                           verbose=verbose)

extent, nrow, ncol = nlmod.read.regis.fit_extent_to_regis(extent,
                                                          delr,
                                                          delc, verbose=verbose)

# layer model
layer_model = nlmod.read.regis.get_layer_models(extent, delr, delc,
                                                use_regis=use_regis,
                                                regis_botm_layer=regis_botm_layer,
                                                use_geotop=use_geotop,
                                                cachedir=cachedir2,
                                                fname_netcdf='combined_layer_ds.nc',
                                                use_cache=use_cache,
                                                verbose=verbose)


creating modflow SIM, TDIS, GWF and IMS
redefining current extent: [95000.0, 105000.0, 494000.0, 500000.0], fit to regis raster
new extent is [94950.0, 105050.0, 493950.0, 500050.0] model has 61 rows and 101 columns
creating and caching dataset combined_layer_ds.nc



In [4]:
%pdb

Automatic pdb calling has been turned ON


In [5]:
# use gridgen to create unstructured grid
gridprops = nlmod.mgrid.create_unstructured_grid(gridgen_ws, model_name, gwf2,
                                                 refine_shp_fname, levels, extent,
                                                 layer_model.dims['layer'],
                                                 nrow, ncol,
                                                 delr, delc,
                                                 cachedir=cachedir2, use_cache=use_cache,
                                                 verbose=verbose)

# add layer model to unstructured grid
layer_model_unstr = nlmod.mgrid.get_ml_layer_dataset_unstruc(raw_ds=layer_model,
                                                            extent=extent,
                                                            gridprops=gridprops,
                                                            cachedir=cachedir2,
                                                            fname_netcdf='layer_model_unstr.nc',
                                                            use_cache=use_cache,
                                                            verbose=verbose)

# combine model time dataset with layer model dataset
model_ds2 = nlmod.mgrid.update_model_ds_from_ml_layer_ds(model_ds2,
                                                          layer_model_unstr,
                                                          gridtype,
                                                          keep_vars=['x', 'y'],
                                                          gridprops=gridprops,
                                                          add_northsea=add_northsea,
                                                          verbose=verbose)

# Create discretization
disv = nlmod.mfpackages.disv_from_model_ds(model_ds2, gwf, gridprops,
                                        angrot=angrot,
                                        length_units=length_units)

create unstructured grid using gridgen


Exception: Cannot find gridgen binary executable

> c:\anaconda3\envs\nhflopy\lib\site-packages\flopy\utils\gridgen.py(224)__init__()
    222         exe_name = which(exe_name)
    223         if exe_name is None:
--> 224             raise Exception("Cannot find gridgen binary executable")
    225         self.exe_name = os.path.abspath(exe_name)
    226 

ipdb> u
> c:\users\oebbe\02_python\nlmod\nlmod\mgrid\mgrid.py(673)create_unstructured_grid()
    671         exe_name = os.path.join(os.path.dirname(__file__),
    672                                 '.', ' .', 'executables', 'gridgen.exe')
--> 673     g = Gridgen(_dis_temp, model_ws=gridgen_ws, exe_name=exe_name)
    674 
    675     g.add_refinement_features(shp_fname, shp_type, levels, range(nlay))

ipdb> exe_name
'c:\\users\\oebbe\\02_python\\nlmod\\nlmod\\mgrid\\.\\ .\\executables\\gridgen.exe'
ipdb> os.path.exists(exe_name)
False
ipdb> os.path.isfile(exe_name)
False
ipdb> q


In [ ]:
# create node property flow
nlmod.mfpackages.npf_from_model_ds(model_ds2, gwf2)

# Create the initial conditions package
nlmod.mfpackages.ic_from_model_ds(model_ds2, gwf2, starting_head=starting_head)

# Create the output control package
nlmod.mfpackages.oc_from_model_ds(model_ds2, gwf2)

In [ ]:
# voeg grote oppervlaktewaterlichamen toe
da_name = 'surface_water'
model_ds2 = nlmod.mfpackages.surface_water.get_general_head_boundary(model_ds2,
                                                                     gwf2.modelgrid,
                                                                     da_name,
                                                                     cachedir=cachedir2,
                                                                     use_cache=use_cache,
                                                                     verbose=verbose)
ghb = nlmod.mfpackages.ghb_from_model_ds(model_ds2, gwf2, da_name)

# surface level drain
model_ds2 = nlmod.read.ahn.get_ahn_dataset(model_ds2, use_cache=use_cache,
                                           cachedir=cachedir2, verbose=verbose)

drn = nlmod.mfpackages.surface_drain_from_model_ds(model_ds2, gwf2)



# add constant head cells at model boundaries
chd = nlmod.mfpackages.chd_at_model_edge_from_model_ds(model_ds2, gwf2, head='starting_head')

In [ ]:
# save model_ds
model_ds2.to_netcdf(os.path.join(cachedir2, 'full_model_ds.nc'))

In [ ]:
# plot modelgrid
ax = nlmod.plots.plot_modelgrid(model_ds2, gwf2)
gpd.read_file(refine_shp+'.shp').plot(ax=ax, color='r', label='Planetenweg')
ax.legend()
ax.get_figure().savefig(os.path.join(figdir2, 'mgrid_swater.png'), bbox_inches='tight')

# plot zoomed modelgrid
ax = nlmod.plots.plot_modelgrid(model_ds2, gwf2)
ax.set_title('Planetenweg')
gpd.read_file(refine_shp+'.shp').plot(ax=ax, color='r', label='Planetenweg')
ax.set_xlim(100000, 103000)
ax.set_ylim(495000,497500)
ax.legend()
ax.get_figure().savefig(os.path.join(figdir2, 'Planetenweg.png'), bbox_inches='tight')

The model dataset of an unstructured model differs from a structured model dataset. The data is stored relative to the cell-id instead of the row and column number. Therefore the model dataset has the dimension cid instead of the dimensions x and y. 

In [ ]:
model_ds2

To plot the same rasters as for the previous model we can use the `nlmod.plots.plot_array()` function.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14,11))
model_ds['ahn'].plot(ax=axes[0][0])
model_ds['bot'][0].plot(ax=axes[0][1])
model_ds['idomain'][0].plot(ax=axes[1][0])
model_ds['chd'][0].plot(ax=axes[1][1])
for axes1 in axes:
    for ax in axes1:
        ax.axis('scaled')

fig.savefig(os.path.join(figdir, 'ahn_bot_idom_chd.png'), bbox_inches='tight')        
        
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14,11))
model_ds['bathymetry'].plot(ax=axes[0][0])
model_ds['northsea'].plot(ax=axes[0][1])
model_ds['kh'][1].plot(ax=axes[1][0])
model_ds['top'].plot(ax=axes[1][1])

for axes1 in axes:
    for ax in axes1:
        ax.axis('scaled')
fig.savefig(os.path.join(figdir, 'bath_nsea_kh_rch.png'), bbox_inches='tight')        

### 4. Local grid refinement<a name="local"></a>

Modflow 6 makes it possible to use locally refined grids. In NLmod you can use a shapefile and a number of levels to specify where and how much you want to use local grid refinement. Below we use a shapefile of the Planetenweg in IJmuiden and set the refinement levels at 2. This well create a grid with cells of 100x100m except at the Planetenweg where the cells will be refined to 25x25m. See also figures below.

Note: the `gen_model_unstructured` function also returns the `gridprops` dictionary. This contains data that is necesary to create some packages. It can be time consuming to generate this data.

In [ ]:
refine_shp = os.path.join(nlmod.nlmod_datadir, 'shapes', 'planetenweg_ijmuiden')

model_ds2, gwf2, gridprops2 = nlmod.create_model.gen_model_unstructured('model2', 'IJm_planeten',
                                                                         use_cache=False,
                                                                         refine_shp_fname=refine_shp,
                                                                         levels=2,
                                                                         delr=100., delc=100.,
                                                                         extent=[95000., 105000., 494000., 500000.],
                                                                         run_sim=True, write_sim=True)

In [ ]:
# get figdir to save figures
figdir2, _, _=nlmod.util.get_model_dirs(model_ds2.model_ws,gridtype=model_ds2.gridtype)

In [ ]:
# plot modelgrid
ax = nlmod.plots.plot_modelgrid(model_ds2, gwf2)
gpd.read_file(refine_shp+'.shp').plot(ax=ax, color='r', label='Planetenweg')
ax.legend()
ax.get_figure().savefig(os.path.join(figdir2, 'mgrid_swater.png'), bbox_inches='tight')

# plot zoomed modelgrid
ax = nlmod.plots.plot_modelgrid(model_ds2, gwf2)
ax.set_title('Planetenweg')
gpd.read_file(refine_shp+'.shp').plot(ax=ax, color='r', label='Planetenweg')
ax.set_xlim(100000, 103000)
ax.set_ylim(495000,497500)
ax.legend()
ax.get_figure().savefig(os.path.join(figdir2, 'Planetenweg.png'), bbox_inches='tight')

The model dataset of an unstructured model differs from a structured model dataset. The data is stored relative to the cell-id instead of the row and column number. Therefore the model dataset has the dimension cid instead of the dimensions x and y. 

In [ ]:
model_ds2

To plot the same rasters as for the previous model we can use the `nlmod.plots.plot_array()` function.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14,11))
nlmod.plots.plot_array(gwf2, model_ds2['ahn'], ax=axes[0][0])
nlmod.plots.plot_array(gwf2, model_ds2['bot'][0], ax=axes[0][1])
nlmod.plots.plot_array(gwf2, model_ds2['idomain'][0], ax=axes[1][0])
nlmod.plots.plot_array(gwf2, model_ds2['chd'][0], ax=axes[1][1])

fig.savefig(os.path.join(figdir2, 'ahn_bot_idom_chd.png'), bbox_inches='tight')        
        
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14,11))
nlmod.plots.plot_array(gwf2, model_ds2['bathymetry'],ax=axes[0][0])
nlmod.plots.plot_array(gwf2, model_ds2['northsea'],ax=axes[0][1])
nlmod.plots.plot_array(gwf2, model_ds2['kh'][1],ax=axes[1][0])
nlmod.plots.plot_array(gwf2, model_ds2['top'],ax=axes[1][0])

fig.savefig(os.path.join(figdir2, 'bath_nsea_kh_top.png'), bbox_inches='tight')        